In [3]:
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"/Users/kaka/Downloads/instantclient_18_1")
from sqlalchemy import create_engine
import json
import pandas as pd
import os
import sqlalchemy

In [4]:
username = 'nppbuf'  
password = 'Svwnppbuf321'  
hostname = '10.122.6.59'  
port = 1521  
service_name = 'b2bt'
dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
# 连接 Oracle 数据库  
connection = cx_Oracle.connect(user=username, password=password, dsn=dsn)
# 定义 SQL 查询  
sql_query = """  
     select substr(hostpartchainid, 1, 4), --工厂
       substr(hostpartchainid, 6),    --系列最终件
       hostpartid,                    --被替代件
       hostchainparentid              --替代件
  from nppbuf.t_dd_part_chain_details
 where hostpartchainid not like '%RDC'
   AND hostpartid != hostchainparentid
"""
# 执行 SQL 查询，并按批次抽取数据  
chunksize = 100000  # 每次查询返回的行数，你可以根据需要调整这个值  
query_chunks = pd.read_sql(sql_query, connection, chunksize=chunksize)
# 遍历查询结果的每个批次，并将其合并为一个 DataFrame  
result = pd.DataFrame()  
for chunk in query_chunks: 
    print(chunk.shape)
    result = result.append(chunk)
# 关闭数据库连接  
connection.close()
# 输出合并后的 DataFrame  
result

/Users/kaka/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


(21150, 4)


/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_52971/664224659.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(chunk)


,"SUBSTR(HOSTPARTCHAINID,1,4)","SUBSTR(HOSTPARTCHAINID,6)",HOSTPARTID,HOSTCHAINPARENTID
0,1000,3TD881347 9B9,3TD881347,3TD881347 9B9
1,1000,330853555 092,330853555,330853555 092
2,1000,3BD858757B,3U0858757D,3BD858757B
3,1000,5ED853651A FOD,5ED853651A F0D,5ED853651A FOD
4,1000,3BD971279A,3BD971279,3BD971279A
...,...,...,...,...
21145,1000,330919051C,330919051A,330919051C
21146,1000,06B105561B 007,06B105561 007,06B105561B 007
21147,1000,5Q0953513R IGI,5Q0953513D IGI,5Q0953513R IGI
21148,1000,5ND837973 82V,5N0837973C 82V,5ND837973 82V


In [7]:
result.columns =['工厂','系列最终件', '被替代件', '替代件']
result.head()

,工厂,系列最终件,被替代件,替代件
0,1000,3TD881347 9B9,3TD881347,3TD881347 9B9
1,1000,330853555 092,330853555,330853555 092
2,1000,3BD858757B,3U0858757D,3BD858757B
3,1000,5ED853651A FOD,5ED853651A F0D,5ED853651A FOD
4,1000,3BD971279A,3BD971279,3BD971279A


In [87]:
test = result[['工厂','被替代件']].head(20)

In [91]:

# 创建一个新的 dataframe，用于存储修正后的 b 数据  
import pandas as pd

merged_df = pd.merge(test, result, on=['工厂', '被替代件'], how='left')

# 使用apply函数根据条件更新零件号列
def update_part_number(row):
    if not pd.isna(row['系列最终件']):
        return row['系列最终件']
    else:
        return row['被替代件']

merged_df['被替代件_02'] = merged_df.apply(update_part_number, axis=1)

# 最终的DataFrame merged_df 包含了修正后的零件号


In [92]:
merged_df

,工厂,被替代件,系列最终件,替代件,被替代件_02
0,1000,3TD881347,3TD881347 9B9,3TD881347 9B9,3TD881347 9B9
1,1000,330853555,330853555 092,330853555 092,330853555 092
2,1000,3U0858757D,3BD858757B,3BD858757B,3BD858757B
3,1000,5ED853651A F0D,5ED853651A FOD,5ED853651A FOD,5ED853651A FOD
4,1000,3BD971279,3BD971279A,3BD971279A,3BD971279A
5,1000,5TD857003C EPG,5TD857003D EPG,5TD857003D EPG,5TD857003D EPG
6,1000,000979308,000979308E,000979308E,000979308E
7,1000,6RD199517,6Q0199517A,6Q0199517A,6Q0199517A
8,1000,180927803,1JD927803,1JD927803,1JD927803
9,1000,3BD839471,3BD839471A B41,3BD839471A B41,3BD839471A B41
